In [ ]:
from src.main import run
from rich import print

## Задание 1 – Проеĸтирование архитеĸтуры

### Агенты и их ответственность
В рамках задания была реализована МАС, которая помогает решать учебные вопросы. В системе используется несколько специализированных агентов, оформленных как узлы графа StateGraph LangGraph.
​
* **RouterAgent**

Классифицирует входной запрос пользователя по категориям: academic, programming, planning, other.

Записывает принятое решение в state["category"] и лог в state["agent_log"]["router"].

* **DecompozerAgent**

Отвечает за декомпозицию программных запросов на подзадачи и формирование execution_plan.

Работает только для запросов категории programming, так как вызывается после маршрутизации.

* **CodeAssistantAgent**

Решает практические программные задачи на основе query и ранее построенного execution_plan.

Использует tool calling для вызова утилитарных инструментов (через get_available_tools, apply_tools, run_tool, extract_tool_calls), а результаты инструментов используются в финальном ответе.
​

* **StudyAssistantAgent**

Обрабатывает теоретические и академические запросы (academic), используя профильные заметки пользователя.

Передает в LLM строку memory_str, собранную из релевантных profile_notes, и тоже может вызывать инструменты через apply_tools.

* **PlannerAgent**

Строит планы и расписания для запросов категории planning.

Получает в prompt предзагруженные profile_notes (например, события пользователя) и использует инструменты для уточнения/обогащения плана.

### Используемые паттерны МАС
Архитектура комбинирует несколько паттернов:
​
* Router + специализированные агенты

**RouterAgent** реализует шаг маршрутизации, выбирая, куда отправить запрос: в **StudyAssistantAgent (академика)**, связку **DecompozerAgent** + **CodeAssistantAgent (программирование)** или **PlannerAgent (планирование)**.

* Planner–Executor для программных задач

Для категории programming сначала вызывается **DecompozerAgent**, который строит execution_plan.
Затем выполняется **CodeAssistantAgent**, который действует как executor, опираясь на план и вызывая инструменты по мере необходимости.


### Tool calling
Tool calling централизованно реализован на уровне специализированных агентов через вспомогательные функции из utils.
​

* Доступные инструменты

Список доступных инструментов формируется функцией get_available_tools() и прокидывается в промпты **CODE_ASSISTANT_PROMPT**, **STUDY_ASSISTANT_PROMPT**, **PLANNER_PROMPT** в виде текстового описания.

* Исполнение инструментов

После генерации ответа агента строки проходят через apply_tools(result).
apply_tools извлекает tool‑call‑маркеры, синхронно вызывает соответствующие Python‑функции через run_tool, собирает их результаты и подставляет их обратно в текст ответа (processed), а также возвращает структуру с логом вызванных инструментов (executed).

Каждый профильный агент логирует свои вызовы в state["agent_log"][f"{agent_name}_tools"], что упрощает отладку и трассировку.

### Управление памятью
Память вынесена в отдельный класс Memory и разделена на историю диалогов и профильные заметки, что соответствует распространённым рекомендациям по организации памяти агентов.
​

* Структура хранилища

Файл memory.json содержит два основных раздела:

msg_history — список объектов {user, assistant} для последних взаимодействий.

profile_notes — список заметок вида {title, content}, описывающих долгосрочные предпочтения и события пользователя.

* История взаимодействий

В run() после завершения графа вызывается memory.update_history(query, result["final_answer"]), что добавляет в msg_history новый диалоговый шаг.

Метод get_history(n) позволяет при необходимости восстановить последние n сообщений и использовать их в будущем как контекст.

* Профильные заметки и их влияние

В начале run() загружается profile_notes = memory.get_from_profile("event"), которые затем передаются в конструктор **PlannerAgent** и попадают в prompt через partial_variables={"profile_notes": str(self.profile_notes)}.

StudyAssistantAgent при каждом запуске делает self.memory.get_from_profile(state["query"]), вычисляет релевантность заметок по совпадениям в title/content и формирует строку memory_str для промпта.

Таким образом, профильные заметки влияют на последующие шаги:

- Планировщик учитывает будущие события и ограничения пользователя.

- Учебный ассистент подстраивает объяснения под уже изученные темы и прошлые конспекты.

In [ ]:
                         +---------------------------+
                         |        User (query)       |
                         +-------------+-------------+
                                       |
                                       v
                          run(query, memory_path)
                                       |
                                       v
                         +---------------------------+
                         |   Init State + Memory     |
                         |  query, category=None,    |
                         |  memory=Memory,           |
                         |  execution_plan=None,     |
                         |  profile_notes, logs      |
                         +-------------+-------------+
                                       |
                                       v
                         +---------------------------+
                         |     RouterAgent node      |
                         |  (classify: category)     |
                         +-------------+-------------+
                                       |
             +-------------------------+--------------------------+
             |                         |                          |
             v                         v                          v
   category = "academic"      category = "programming"   category = "planning"
             |                         |                          |
             v                         v                          v
+------------------------+   +------------------------+   +------------------------+
|  StudyAssistantAgent   |   |    DecompozerAgent     |   |     PlannerAgent       |
|  (uses profile_memory) |   |  (build execution_plan)|   | (uses profile_notes,   |
|  + tools via           |   |                        |   |  tools via apply_tools)|
|    get_available_tools |   +-----------+------------+   +-----------+------------+
+-----------+------------+               |                            |
            |                            v                            |
            |                 +------------------------+              |
            |                 |   CodeAssistantAgent  |               |
            |                 | (use execution_plan   |               |
            |                 |  + tools via          |               |
            |                 |  apply_tools)         |               |
            |                 +-----------+-----------+               |
            |                             |                           |
            +-----------------------------+---------------------------+
                                          |                             
                                          v                             
                            +---------------------------+ 
                            |     Update state:         |
                            |  final_answer from        |
                            |  agent_log[...]           |
                            +-------------+-------------+
                                          |
                                          v
                            +---------------------------+
                            |  memory.update_history()  |
                            |  (msg_history in JSON)    |
                            +-------------+-------------+
                                          |
                                          v
                            +---------------------------+
                            |      Return result        |
                            |   (final_answer, logs,    |
                            |    updated memory)        |
                            +---------------------------+

## Задание 3 – Эĸсперименты и неформальная оценĸа

### Концептуальный/теоретичесĸий вопрос про МАС или LLM-агентов

In [11]:
query = "In a multi-agent system composed of LLM-based agents, where does responsibility for decisions actually reside: in individual agents, in the coordination protocol, or in the human operator?"

result = run(query)

In [12]:
print(result["final_answer"])

In a multi-agent system with LLM-based agents, **responsibility for decisions is distributed but ultimately 
anchored in human operators**, with nuanced contributions from the agents and coordination protocols. Here's a 
structured breakdown:

---

### 1. **Individual Agents**
   - **Role**: Execute decisions based on their training data, prompts, and internal logic (e.g., choosing actions,
responding to queries).
   - **Limitations**: 
     - LLM agents lack intent, consciousness, or moral agency. Their outputs are probabilistic and derived from 
patterns in training data.
     - They cannot be held accountable for decisions (e.g., biased outputs or errors) because they do not 
"understand" consequences.
   - **Responsibility**: 
     - **Designers/Trainers**: Responsible for biases, capabilities, and limitations in the agent (e.g., if an 
agent generates harmful content due to training data).
     - **Deployers**: Must ensure agents are used appropriately for their intended tasks.

---

### 2. **Coordination Protocol**
   - **Role**: Defines rules for agent interaction (e.g., consensus algorithms, communication formats, conflict 
resolution).
   - **Responsibility**:
     - **Designers**: If the protocol leads to failures (e.g., deadlock, inefficient resource allocation), the 
fault lies in its design.
     - Example: A poorly designed voting protocol might cause agents to make suboptimal group decisions, even if 
individual agents act correctly.
   - **Limitation**: The protocol itself is a passive mechanism; it does not "decide" but structures interactions.

---

### 3. **Human Operator**
   - **Role**: Oversees the system, defines goals, and intervenes when needed.
   - **Key Responsibilities**:
     1. **System Design**: Choosing agents, protocols, and environments.
     2. **Ethical Oversight**: Ensuring the system aligns with human values (e.g., fairness, safety).
     3. **Accountability**: Legally and ethically responsible for outcomes (e.g., if an autonomous trading system 
causes financial harm).
   - **Example**: In a self-driving car system, the manufacturer (human operator) is liable for accidents caused by
flaws in the AI agents or their coordination.

---

### 4. **Shared Responsibility Framework**
   - **Causal Chains**: 
     - An agent’s decision might stem from its training (designer), the protocol’s constraints (protocol designer),
and the operator’s deployment choices.
   - **Ethical Lens**: 
     - Since LLMs lack moral agency, responsibility "trickles up" to humans. For instance, if an agent 
discriminates, the responsibility falls on the trainer (for biased data) and the operator (for inadequate 
safeguards).
   - **Legal Reality**: 
     - Laws typically assign liability to organizations or individuals, not AI systems (e.g., EU AI Act proposals).

---

### Summary Table
| Component               | Can "Decide"? | Bears Responsibility? | Notes                                      |
|-------------------------|---------------|-----------------------|--------------------------------------------|
| **LLM Agents**          | Yes (statistically) | No                   | Outputs are tools, not autonomous actors. 
|
| **Coordination Protocol** | No          | Yes (via designers)   | Failures reflect poor design.              |
| **Human Operator**      | Indirectly    | Yes (ultimately)      | Legal/ethical accountability is assigned.  |

---

### Teaching Analogy
Think of a **robotic orchestra**:
- **Agents** (robots) play instruments based on their programming (training).
- The **conductor’s protocol** dictates how they synchronize (e.g., tempo, cues).
- The **human conductor** is responsible for the performance’s success or failure, even if a robot misplays a note 
due to a design flaw or ambiguous instructions.

In this analogy, the robots (LLM agents) and score (protocol) are tools shaped by human intent and oversight.

---

By understanding this layered structure, you can analyze real-world systems (e.g

In [13]:
print(result["agent_log"])

{
    'router': '\n\nclassification: academic',
    'study_assistant': '\n\nIn a multi-agent system with LLM-based agents, **responsibility for decisions is 
distributed but ultimately anchored in human operators**, with nuanced contributions from the agents and 
coordination protocols. Here\'s a structured breakdown:\n\n---\n\n### 1. **Individual Agents**\n   - **Role**: 
Execute decisions based on their training data, prompts, and internal logic (e.g., choosing actions, responding to 
queries).\n   - **Limitations**: \n     - LLM agents lack intent, consciousness, or moral agency. Their outputs are
probabilistic and derived from patterns in training data.\n     - They cannot be held accountable for decisions 
(e.g., biased outputs or errors) because they do not "understand" consequences.\n   - **Responsibility**: \n     - 
**Designers/Trainers**: Responsible for biases, capabilities, and limitations in the agent (e.g., if an agent 
generates harmful content due to training data).\n     - **Deployers**: Must ensure agents are used appropriately 
for their intended tasks.\n\n---\n\n### 2. **Coordination Protocol**\n   - **Role**: Defines rules for agent 
interaction (e.g., consensus algorithms, communication formats, conflict resolution).\n   - **Responsibility**:\n  
- **Designers**: If the protocol leads to failures (e.g., deadlock, inefficient resource allocation), the fault 
lies in its design.\n     - Example: A poorly designed voting protocol might cause agents to make suboptimal group 
decisions, even if individual agents act correctly.\n   - **Limitation**: The protocol itself is a passive 
mechanism; it does not "decide" but structures interactions.\n\n---\n\n### 3. **Human Operator**\n   - **Role**: 
Oversees the system, defines goals, and intervenes when needed.\n   - **Key Responsibilities**:\n     1. **System 
Design**: Choosing agents, protocols, and environments.\n     2. **Ethical Oversight**: Ensuring the system aligns 
with human values (e.g., fairness, safety).\n     3. **Accountability**: Legally and ethically responsible for 
outcomes (e.g., if an autonomous trading system causes financial harm).\n   - **Example**: In a self-driving car 
system, the manufacturer (human operator) is liable for accidents caused by flaws in the AI agents or their 
coordination.\n\n---\n\n### 4. **Shared Responsibility Framework**\n   - **Causal Chains**: \n     - An agent’s 
decision might stem from its training (designer), the protocol’s constraints (protocol designer), and the 
operator’s deployment choices.\n   - **Ethical Lens**: \n     - Since LLMs lack moral agency, responsibility 
"trickles up" to humans. For instance, if an agent discriminates, the responsibility falls on the trainer (for 
biased data) and the operator (for inadequate safeguards).\n   - **Legal Reality**: \n     - Laws typically assign 
liability to organizations or individuals, not AI systems (e.g., EU AI Act proposals).\n\n---\n\n### Summary 
Table\n| Component               | Can "Decide"? | Bears Responsibility? | Notes                                   
|\n|-------------------------|---------------|-----------------------|--------------------------------------------|
\n| **LLM Agents**          | Yes (statistically) | No                   | Outputs are tools, not autonomous 
actors.  |\n| **Coordination Protocol** | No          | Yes (via designers)   | Failures reflect poor design.      
|\n| **Human Operator**      | Indirectly    | Yes (ultimately)      | Legal/ethical accountability is assigned.  
|\n\n---\n\n### Teaching Analogy\nThink of a **robotic orchestra**:\n- **Agents** (robots) play instruments based 
on their programming (training).\n- The **conductor’s protocol** dictates how they synchronize (e.g., tempo, 
cues).\n- The **human conductor** is responsible for the performance’s success or failure, even if a robot misplays
a note due to a design flaw or ambiguous instructions.\n\nIn this analogy, the robots (LLM agents) and score 
(p

Для ответа на теоретический вопрос Router верно определил необходимость передачи запроса агенту StudyAssistant (academic). Сам же агент никаких тулов для выполения запроса не вызвал, поскольку требовался исключительно теоретический ответ, не предполагающий вызова ни одного из доступных тулов.

### Вопрос по проеĸтированию/архитеĸтуре

In [14]:
query = "Design a high-level architecture for a multi-agent study assistant that can route queries, decompose tasks, call tools, and maintain user-specific memory across sessions."

result = run(query)

In [15]:
print(result["final_answer"])

Here's a structured high-level architecture for a multi-agent study assistant, explained step-by-step with 
teaching-focused clarity:

---

### **Multi-Agent Study Assistant Architecture**
This system combines **modular agents** with **persistent memory** to handle complex academic tasks. Let's break it
down:

---

#### **1. Core Components**
**A. User Interface (UI) Layer**
- **Purpose**: Accepts user input (text/voice) and displays outputs.
- **Example**: A chat interface where students ask questions like, "How do I solve this calculus problem?"

**B. Router Agent**
- **Role**: First point of contact. Analyzes the query's intent using NLP (e.g., math, coding, theory) and routes 
it to the appropriate specialist agent.
- **Example**: If the user asks, "Validate this Python code," the router directs it to the **Code Validation 
Agent**.

**C. Specialist Agents**
- **Task-Specific Roles**:
  - **Math Agent**: Solves equations using a calculator tool (e.g., `sin(pi/2)`).
  - **Code Agent**: Uses `validate_code` or `safe_execute` for programming tasks.
  - **Theory Agent**: Explains concepts (e.g., "What is Ohm's Law?").
- **Why it works**: Modular design allows each agent to focus on its domain, improving accuracy.

**D. Task Decomposition Agent**
- **Role**: Breaks complex tasks into smaller sub-tasks.
- **Example**: For "Design an experiment on photosynthesis," it might split into:
  1. Explain photosynthesis (Theory Agent).
  2. Suggest materials (Science Agent).
  3. Format results (Writing Agent).

**E. Memory Manager Agent**
- **Purpose**: Stores/retrieves user-specific data across sessions (e.g., past queries, preferences, progress).
- **Implementation**: Uses a database (e.g., user ID → study history).
- **Example**: Remembers a student's struggle with quadratic equations to prioritize practice problems.

**F. Tool Interfacing Agents**
- **Purpose**: Act as intermediaries between agents and tools (e.g., `calculator`, `validate_code`).
- **Why needed**: Ensures tools are called securely and inputs/outputs are properly formatted.

**G. Orchestrator Agent**
- **Role**: Coordinates communication between agents, ensuring tasks complete in the correct order.
- **Example**: If the Code Agent needs a math result first, the orchestrator waits for the Math Agent's output.

**H. Response Synthesizer**
- **Purpose**: Combines results from multiple agents into a coherent answer for the user.

---

#### **2. Data Flow Example**
**User Query**: "Solve 3 + 5² and explain the steps."
1. **Router** directs to **Math Agent**.
2. **Math Agent** sends `3 + 5^2` to the **Calculator Interfacing Agent**.
3. **Calculator** returns `28`.
4. **Theory Agent** explains the order of operations (PEMDAS).
5. **Response Synthesizer** combines the result and explanation.
6. **Memory Manager** logs the interaction under the user's session.

---

#### **3. Key Design Principles**
- **Modularity**: Agents can be updated/added independently (e.g., add a "History Agent" later).
- **Persistence**: Memory Manager ensures personalized, continuity-rich interactions.
- **Scalability**: New tools or agents can integrate without disrupting existing workflows.

---

#### **4. Challenges & Solutions**
- **Challenge**: Ambiguous queries (e.g., "Help with this").
  - **Solution**: Router uses intent classification models or asks clarifying questions.
- **Challenge**: Tool errors (e.g., invalid syntax).
  - **Solution**: Tool agents return structured error messages for the orchestrator to handle.
- **Challenge**: Privacy.
  - **Solution**: Memory Manager anonymizes data unless the user consents to tracking.

---

#### **5. Teaching Analogy**
Think of the system as a **school with specialized teachers**:
- The **principal (Router)** assigns students to the right teacher.
- The **math teacher** uses a **calculator (Tool)** for problems.
- The **homeroom teacher (Memory Manager)** knows each student's progress.
- The **dean (Orchestrator)** ensures everyone works together s

In [16]:
print(result["agent_log"])

{
    'router': '\n\nclassification: academic',
    'study_assistant': '\n\nHere\'s a structured high-level architecture for a multi-agent study assistant, 
explained step-by-step with teaching-focused clarity:\n\n---\n\n### **Multi-Agent Study Assistant 
Architecture**\nThis system combines **modular agents** with **persistent memory** to handle complex academic 
tasks. Let\'s break it down:\n\n---\n\n#### **1. Core Components**\n**A. User Interface (UI) Layer**\n- 
**Purpose**: Accepts user input (text/voice) and displays outputs.\n- **Example**: A chat interface where students 
ask questions like, "How do I solve this calculus problem?"\n\n**B. Router Agent**\n- **Role**: First point of 
contact. Analyzes the query\'s intent using NLP (e.g., math, coding, theory) and routes it to the appropriate 
specialist agent.\n- **Example**: If the user asks, "Validate this Python code," the router directs it to the 
**Code Validation Agent**.\n\n**C. Specialist Agents**\n- **Task-Specific Roles**:\n  - **Math Agent**: Solves 
equations using a calculator tool (e.g., `sin(pi/2)`).\n  - **Code Agent**: Uses `validate_code` or `safe_execute` 
for programming tasks.\n  - **Theory Agent**: Explains concepts (e.g., "What is Ohm\'s Law?").\n- **Why it works**:
Modular design allows each agent to focus on its domain, improving accuracy.\n\n**D. Task Decomposition Agent**\n- 
**Role**: Breaks complex tasks into smaller sub-tasks.\n- **Example**: For "Design an experiment on 
photosynthesis," it might split into:\n  1. Explain photosynthesis (Theory Agent).\n  2. Suggest materials (Science
Agent).\n  3. Format results (Writing Agent).\n\n**E. Memory Manager Agent**\n- **Purpose**: Stores/retrieves 
user-specific data across sessions (e.g., past queries, preferences, progress).\n- **Implementation**: Uses a 
database (e.g., user ID → study history).\n- **Example**: Remembers a student\'s struggle with quadratic equations 
to prioritize practice problems.\n\n**F. Tool Interfacing Agents**\n- **Purpose**: Act as intermediaries between 
agents and tools (e.g., `calculator`, `validate_code`).\n- **Why needed**: Ensures tools are called securely and 
inputs/outputs are properly formatted.\n\n**G. Orchestrator Agent**\n- **Role**: Coordinates communication between 
agents, ensuring tasks complete in the correct order.\n- **Example**: If the Code Agent needs a math result first, 
the orchestrator waits for the Math Agent\'s output.\n\n**H. Response Synthesizer**\n- **Purpose**: Combines 
results from multiple agents into a coherent answer for the user.\n\n---\n\n#### **2. Data Flow Example**\n**User 
Query**: "Solve 3 + 5² and explain the steps."\n1. **Router** directs to **Math Agent**.\n2. **Math Agent** sends 
`3 + 5^2` to the **Calculator Interfacing Agent**.\n3. **Calculator** returns `28`.\n4. **Theory Agent** explains 
the order of operations (PEMDAS).\n5. **Response Synthesizer** combines the result and explanation.\n6. **Memory 
Manager** logs the interaction under the user\'s session.\n\n---\n\n#### **3. Key Design Principles**\n- 
**Modularity**: Agents can be updated/added independently (e.g., add a "History Agent" later).\n- **Persistence**: 
Memory Manager ensures personalized, continuity-rich interactions.\n- **Scalability**: New tools or agents can 
integrate without disrupting existing workflows.\n\n---\n\n#### **4. Challenges & Solutions**\n- **Challenge**: 
Ambiguous queries (e.g., "Help with this").\n  - **Solution**: Router uses intent classification models or asks 
clarifying questions.\n- **Challenge**: Tool errors (e.g., invalid syntax).\n  - **Solution**: Tool agents return 
structured error messages for the orchestrator to handle.\n- **Challenge**: Privacy.\n  - **Solution**: Memory 
Manager anonymizes data unless the user consents to tracking.\n\n---\n\n#### **5. Teaching Analogy**\nThink of the 
system as a **school with specialized teachers**:\n- The **principal (Router)** assigns students to the right 
teacher.\n- The **ma

Для вопроса по проектированию/архитектуре Router также верно назначил агента StudyAssistant (acdemic), поскольку вопрос снова больше теоретический и не требует вызова узконаправленных агентов или использования дополнительных тулов.

### Вопрос по реализации/программированию

In [17]:
query = "Implement a Python class that orchestrates multiple LLM agents (router, decomposer, code assistant) using LangGraph, including shared state and tool execution."

result = run(query)

In [18]:
print(result["final_answer"])

Here's a production-ready implementation using LangGraph with the specified agents, shared state, and tool 
integration:

```python
from langgraph.graph import StateGraph, Graph
from typing import Dict, List, Optional, Tuple, Any
import uuid
import time
import threading

class SharedState:
    def __init__(self):
        self.version = 0
        self.task = ""
        self.subtasks = []
        self.code_results = []
        self.error_log = []
        self.status = "initialized"
        self.lock = threading.Lock()
        
    def update(self, **kwargs):
        with self.lock:
            self.version += 1
            for k, v in kwargs.items():
                setattr(self, k, v)
            return self.version

    def get_snapshot(self):
        with self.lock:
            return {
                "version": self.version,
                "task": self.task,
                "subtasks": self.subtasks.copy(),
                "code_results": self.code_results.copy(),
                "error_log": self.error_log.copy(),
                "status": self.status
            }

def router_agent(state: SharedState) -> Tuple[Dict, str]:
    """Route between decomposer and code assistant based on task type"""
    if not state.subtasks and state.status == "initialized":
        # Need decomposition
        return state.get_snapshot(), "decomposer_agent"
    
    if state.subtasks and any(not subtask.get("completed") for subtask in state.subtasks):
        # Need code execution
        return state.get_snapshot(), "code_assistant_agent"
        
    return state.get_snapshot(), "final"

def decomposer_agent(state: SharedState) -> Dict:
    """Generate subtasks using LLM-style decomposition"""
    try:
        # Mock decomposition logic (replace with actual LLM call)
        state.subtasks = [
            {"id": str(uuid.uuid4()), "description": f"Subtask {i} for {state.task}", 
             "completed": False} for i in range(3)
        ]
        state.status = "decomposed"
        state.update()
        return state.get_snapshot()
    except Exception as e:
        state.error_log.append(f"Decomposition error: {str(e)}")
        return state.get_snapshot()

def code_assistant_agent(state: SharedState) -> Dict:
    """Execute code with tool integration and error handling"""
    try:
        # Find first incomplete subtask
        for subtask in state.subtasks:
            if not subtask["completed"]:
                # Mock code generation (replace with LLM-generated code)
                code = "1 + 2 * 3"
                
                # Validate code
                [Result: ✅ Code is syntactically valid.]
                
                # Execute code with timeout
                result = [Result: ❌ Error: name 'code' is not defined]
                
                state.code_results.append({
                    "subtask_id": subtask["id"],
                    "code": code,
                    "result": result,
                    "timestamp": time.time()
                })
                subtask["completed"] = True
                break
                
        state.status = "executing"
        state.update()
        return state.get_snapshot()
        
    except Exception as e:
        state.error_log.append(f"Execution error: {str(e)}")
        return state.get_snapshot()

def tool_executor(code: str, timeout: int = 5) -> str:
    """Thread-safe tool execution wrapper"""
    result = {"output": "", "error": ""}
    
    def target():
        try:
            result["output"] = [Result: ❌ Error: name 'code' is not defined]
        except Exception as e:
            result["error"] = str(e)
    
    thread = threading.Thread(target=target)
    thread.start()
    thread.join(timeout)
    
    if thread.is_alive():
        return "Execution timeout"
    return result["output"] if not result["error"] else result["error"]

# Graph orchestration
workflow = StateGraph(SharedState)
workflow.add_node("router_agent", router_agent)
workfl

In [19]:
print(result["agent_log"])

{
    'router': '\n\nclassification: programming',
    'execution_plan': "\n\nSubtasks:\n1. Analyze LangGraph documentation to determine node/edge/state 
implementation patterns for multi-agent orchestration.\n2. Design the base graph structure with three nodes: 
RouterAgent, DecomposerAgent, CodeAssistantAgent, and define their input/output interfaces.\n3. Implement a 
SharedState class with versioned state storage, conflict resolution, and change tracking for cross-agent 
communication.\n4. Create the RouterAgent node implementation with decision logic for routing between decomposer 
and code assistant based on task type/state.\n5. Implement the DecomposerAgent node with task analysis capabilities
and subtask generation using LLM prompts.\n6. Develop the CodeAssistantAgent node with code execution integration 
(e.g., subprocess, sandboxed environment) and result parsing.\n7. Implement state synchronization mechanisms 
between nodes using LangGraph's built-in state management features.\n8. Add tool execution wrapper that handles 
code execution requests from CodeAssistantAgent with timeout, error handling, and result capture.\n9. Write 
integration tests for end-to-end workflow: task submission → routing → decomposition → code execution → state 
verification.\n10. Implement error handling and retry logic for failed transitions between agents and failed code 
executions.",
    'code_assistant': '\n\nHere\'s a production-ready implementation using LangGraph with the specified agents, 
shared state, and tool integration:\n\n```python\nfrom langgraph.graph import StateGraph, Graph\nfrom typing import
Dict, List, Optional, Tuple, Any\nimport uuid\nimport time\nimport threading\n\nclass SharedState:\n    def 
__init__(self):\n        self.version = 0\n        self.task = ""\n        self.subtasks = []\n        
self.code_results = []\n        self.error_log = []\n        self.status = "initialized"\n        self.lock = 
threading.Lock()\n        \n    def update(self, **kwargs):\n        with self.lock:\n            self.version += 
1\n            for k, v in kwargs.items():\n                setattr(self, k, v)\n            return 
self.version\n\n    def get_snapshot(self):\n        with self.lock:\n            return {\n                
"version": self.version,\n                "task": self.task,\n                "subtasks": self.subtasks.copy(),\n  
"code_results": self.code_results.copy(),\n                "error_log": self.error_log.copy(),\n                
"status": self.status\n            }\n\ndef router_agent(state: SharedState) -> Tuple[Dict, str]:\n    """Route 
between decomposer and code assistant based on task type"""\n    if not state.subtasks and state.status == 
"initialized":\n        # Need decomposition\n        return state.get_snapshot(), "decomposer_agent"\n    \n    if
state.subtasks and any(not subtask.get("completed") for subtask in state.subtasks):\n        # Need code 
execution\n        return state.get_snapshot(), "code_assistant_agent"\n        \n    return state.get_snapshot(), 
"final"\n\ndef decomposer_agent(state: SharedState) -> Dict:\n    """Generate subtasks using LLM-style 
decomposition"""\n    try:\n        # Mock decomposition logic (replace with actual LLM call)\n        
state.subtasks = [\n            {"id": str(uuid.uuid4()), "description": f"Subtask {i} for {state.task}", \n       
"completed": False} for i in range(3)\n        ]\n        state.status = "decomposed"\n        state.update()\n    
return state.get_snapshot()\n    except Exception as e:\n        state.error_log.append(f"Decomposition error: 
{str(e)}")\n        return state.get_snapshot()\n\ndef code_assistant_agent(state: SharedState) -> Dict:\n    
"""Execute code with tool integration and error handling"""\n    try:\n        # Find first incomplete subtask\n   
for subtask in state.subtasks:\n            if not subtask["completed"]:\n                # Mock code generation 
(replace with LLM-generated code)\n          

Для решения задачи по программированию Router корректно определил, что ее нужно передать агенту CodeAssistant (programming). Сам агент вызвал доступные ему тулы для работы с кодом.

### Запрос 1, связанный с  повседневными задачами

In [20]:
query = "Help me plan my study schedule for the next two weeks to prepare for an NLP exam, given that I can study about 2 hours per day."

result = run(query)

In [21]:
print(result["final_answer"])

Here's a structured 2-week study plan for your NLP exam (2 hours/day), integrated with your schedule constraints:

---

### **Daily Study Blocks (2 hours each)**
**Note:** Adjust timing around events (e.g., study in the morning before evening events).

---

### **Week 1: Foundations & Core Concepts**
**Jan 4 (Thu)**  
- 18:00-20:00: **Intro to NLP**  
  - Define NLP tasks (tokenization, POS tagging, parsing).  
  - Review key libraries: NLTK, spaCy.  

**Jan 5 (Fri)**  
- 18:00-20:00: **Text Preprocessing**  
  - Clean text (stopword removal, stemming, lemmatization).  
  - Practice with code snippets (use `validate_code` for syntax checks).  

**Jan 6 (Sat)**  
- 10:00-12:00: **Word Embeddings**  
  - Study Word2Vec/GloVe mechanics.  
  - Use `calculator` to compute cosine similarity between vectors.  

**Jan 7 (Sun)**  
- 18:00-20:00: **RNNs & LSTMs**  
  - Diagram architectures.  
  - Write pseudocode for sequence modeling.  

**Jan 8 (Mon)**  
- 18:00-20:00: **Transformers & Attention**  
  - Break down self-attention mechanism.  
  - Annotate a transformer block diagram.  

**Jan 9 (Tue)**  
- 10:00-12:00: **Review Gym Day**  
  - Flashcards for NLP terminology.  
  - Quick recap of Week 1 topics.  

**Jan 10 (Wed)**  
- 18:00-20:00: **NLP Applications**  
  - Study NER, sentiment analysis, and QA systems.  
  - Compare rule-based vs. ML approaches.  

---

### **Week 2: Advanced Topics & Practice**
**Jan 11 (Thu)**  
- 18:00-20:00: **Advanced Models**  
  - Study BERT, GPT-3, and fine-tuning.  
  - Use `safe_execute` to test tokenization code.  

**Jan 12 (Fri)**  
- 18:00-20:00: **Practice Coding**  
  - Build a text classifier (scikit-learn or TensorFlow).  
  - Validate syntax with `validate_code`.  

**Jan 13 (Sat)**  
- 10:00-12:00: **Mock Exam 1**  
  - Solve 20 MCQs + 1 coding problem.  
  - Review answers immediately.  

**Jan 14 (Sun)**  
- 18:00-20:00: **Mock Exam 2**  
  - Focus on transformer-based questions.  
  - Time yourself strictly.  

**Jan 15 (Mon)**  
- 18:00-20:00: **Weakness Review**  
  - Revisit topics from mock exams.  
  - Create a cheat sheet for formulas/concepts.  

**Jan 16 (Tue)**  
- 18:00-20:00: **Final Coding Drill**  
  - Implement a seq2seq model (use `calculator` for math checks).  

**Jan 17 (Wed)**  
- 18:00-20:00: **Exam Simulation**  
  - Full-length practice test under exam conditions.  

**Jan 18 (Thu)**  
- 18:00-20:00: **Final Review**  
  - Flashcards + cheat sheet review.  
  - Relax and prepare mentally.  

---

### **Tools Integration Tips**  
- Use `validate_code` for debugging NLP scripts.  
- Use `calculator` for probability/math problems (e.g., perplexity calculations).  
- Use `safe_execute` for quick expression checks (e.g., `len(tokenizer("hello"))`).  

Adjust timing as needed around your events! 🎓

In [22]:
print(result["agent_log"])

{
    'router': '\n\nclassification: planning',
    'planner': '\n\nHere\'s a structured 2-week study plan for your NLP exam (2 hours/day), integrated with your 
schedule constraints:\n\n---\n\n### **Daily Study Blocks (2 hours each)**\n**Note:** Adjust timing around events 
(e.g., study in the morning before evening events).\n\n---\n\n### **Week 1: Foundations & Core Concepts**\n**Jan 4 
(Thu)**  \n- 18:00-20:00: **Intro to NLP**  \n  - Define NLP tasks (tokenization, POS tagging, parsing).  \n  - 
Review key libraries: NLTK, spaCy.  \n\n**Jan 5 (Fri)**  \n- 18:00-20:00: **Text Preprocessing**  \n  - Clean text 
(stopword removal, stemming, lemmatization).  \n  - Practice with code snippets (use `validate_code` for syntax 
checks).  \n\n**Jan 6 (Sat)**  \n- 10:00-12:00: **Word Embeddings**  \n  - Study Word2Vec/GloVe mechanics.  \n  - 
Use `calculator` to compute cosine similarity between vectors.  \n\n**Jan 7 (Sun)**  \n- 18:00-20:00: **RNNs & 
LSTMs**  \n  - Diagram architectures.  \n  - Write pseudocode for sequence modeling.  \n\n**Jan 8 (Mon)**  \n- 
18:00-20:00: **Transformers & Attention**  \n  - Break down self-attention mechanism.  \n  - Annotate a transformer
block diagram.  \n\n**Jan 9 (Tue)**  \n- 10:00-12:00: **Review Gym Day**  \n  - Flashcards for NLP terminology.  \n
- Quick recap of Week 1 topics.  \n\n**Jan 10 (Wed)**  \n- 18:00-20:00: **NLP Applications**  \n  - Study NER, 
sentiment analysis, and QA systems.  \n  - Compare rule-based vs. ML approaches.  \n\n---\n\n### **Week 2: Advanced
Topics & Practice**\n**Jan 11 (Thu)**  \n- 18:00-20:00: **Advanced Models**  \n  - Study BERT, GPT-3, and 
fine-tuning.  \n  - Use `safe_execute` to test tokenization code.  \n\n**Jan 12 (Fri)**  \n- 18:00-20:00: 
**Practice Coding**  \n  - Build a text classifier (scikit-learn or TensorFlow).  \n  - Validate syntax with 
`validate_code`.  \n\n**Jan 13 (Sat)**  \n- 10:00-12:00: **Mock Exam 1**  \n  - Solve 20 MCQs + 1 coding problem.  
\n  - Review answers immediately.  \n\n**Jan 14 (Sun)**  \n- 18:00-20:00: **Mock Exam 2**  \n  - Focus on 
transformer-based questions.  \n  - Time yourself strictly.  \n\n**Jan 15 (Mon)**  \n- 18:00-20:00: **Weakness 
Review**  \n  - Revisit topics from mock exams.  \n  - Create a cheat sheet for formulas/concepts.  \n\n**Jan 16 
(Tue)**  \n- 18:00-20:00: **Final Coding Drill**  \n  - Implement a seq2seq model (use `calculator` for math 
checks).  \n\n**Jan 17 (Wed)**  \n- 18:00-20:00: **Exam Simulation**  \n  - Full-length practice test under exam 
conditions.  \n\n**Jan 18 (Thu)**  \n- 18:00-20:00: **Final Review**  \n  - Flashcards + cheat sheet review.  \n  -
Relax and prepare mentally.  \n\n---\n\n### **Tools Integration Tips**  \n- Use `validate_code` for debugging NLP 
scripts.  \n- Use `calculator` for probability/math problems (e.g., perplexity calculations).  \n- Use 
`safe_execute` for quick expression checks (e.g., `len(tokenizer("hello"))`).  \n\nAdjust timing as needed around 
your events! 🎓',
    'planner_tools': []
}

Поскольку задача подразумевает планирование, Router передает ее агенту Planner (planning). Сам агент корректно отвечает на вопрос не задействуя никаких тулов.

### Запрос 2, связанный с  повседневными задачами

In [6]:
query = "Help me plan a trip to Canada. Do I have any plans on January 6th? If so, suggest better time."

result = run(query)

In [8]:
print(result["final_answer"])

Here's your structured plan:

**Conflict Check:**  
You have a **friend's birthday party on January 6th**, which conflicts with your trip planning.  

**Suggested Alternative Dates:**  
1. **January 7th–8th**: Immediately after your birthday party.  
2. **January 4th–5th**: A few days before the party (if feasible).  
3. **January 14th–16th**: After your family dinner on the 13th.  

**Action Steps to Plan Your Trip:**  
1. **Choose a Date Window** (e.g., Jan 7–8 or Jan 14–16).  
2. **Research Destinations**: Shortlist Canadian cities (e.g., Toronto, Vancouver, Montreal).  
3. **Check Travel Requirements**: Confirm passport/visa validity.  
4. **Book Flights**: Use a travel site (e.g., Google Flights) for the chosen dates.  
5. **Reserve Accommodation**: Book a hotel or Airbnb near your destination.  
6. **Plan Daily Itinerary**: Prioritize attractions (museums, landmarks, outdoor activities).  
7. **Budget**: Allocate funds for food, transport, and activities.  
8. **Packing List**: Prepare weather-appropriate clothing (check January climate for your destination).  

**Next Steps**:  
- Confirm availability for your preferred alternative dates.  
- Use a calendar app to block out trip dates once finalized.  

Let me know if you need adjustments! 🌱

In [9]:
print(result["agent_log"])

{
    'router': '\n\nclassification: planning',
    'planner': "\n\nHere's your structured plan:\n\n**Conflict Check:**  \nYou have a **friend's birthday party on 
January 6th**, which conflicts with your trip planning.  \n\n**Suggested Alternative Dates:**  \n1. **January 
7th–8th**: Immediately after your birthday party.  \n2. **January 4th–5th**: A few days before the party (if 
feasible).  \n3. **January 14th–16th**: After your family dinner on the 13th.  \n\n**Action Steps to Plan Your 
Trip:**  \n1. **Choose a Date Window** (e.g., Jan 7–8 or Jan 14–16).  \n2. **Research Destinations**: Shortlist 
Canadian cities (e.g., Toronto, Vancouver, Montreal).  \n3. **Check Travel Requirements**: Confirm passport/visa 
validity.  \n4. **Book Flights**: Use a travel site (e.g., Google Flights) for the chosen dates.  \n5. **Reserve 
Accommodation**: Book a hotel or Airbnb near your destination.  \n6. **Plan Daily Itinerary**: Prioritize 
attractions (museums, landmarks, outdoor activities).  \n7. **Budget**: Allocate funds for food, transport, and 
activities.  \n8. **Packing List**: Prepare weather-appropriate clothing (check January climate for your 
destination).  \n\n**Next Steps**:  \n- Confirm availability for your preferred alternative dates.  \n- Use a 
calendar app to block out trip dates once finalized.  \n\nLet me know if you need adjustments! 🌱",
    'planner_tools': []
}

Еще одна повседневная задача. Router верно передал ее агенту Planner (planning), никаких тулов данная задача не требует, поэтому здесь они не задействованы. На данном примере можно увидеть, что агент использует информацию полученную из памяти (**friend's birthday party on January 6th** находится в memory.json и агент, имея эту информацию, предлагает альтернативный вариант на запрос пользователя с учетом полученных ограничений).

## Вывод

Таким образом, реализованная МАС хорошо справляется со всеми задачами, на которых она запущена. Агент Router верно распределяет задачи по агентам. Если того требует задача, агенты вызывают необходимые для ее решения тулы. Также как можно видеть на последнем примере, реализация памяти также была задействована в ходе решения задачи. Довольно удобным оказался формат хранения в виде json, где получилось разбить логику хранения логов агентов и какие-то персональные факты о пользователе, однако для более серьезных проектов такой реализации вероятно было бы недостаточно. Также можно было бы добавить агента, который бы валидировал полученный ответ, так как, несмотря на то, что данная МАС отработала хорошо, в более сложных задачах она потенциально может выдавать ответ с ошибками, которые было бы неплохо отлавливать. 